In [1]:
%load_ext autoreload
%autoreload 2
import nnsight
nnsight.__version__

'0.5.0.dev2'

In [1]:
from nnsight.modeling.vllm import VLLM

# NNsight's VLLM wrapper currently supports "device = cuda" and device = "auto"
vllm = VLLM("Maykeye/TinyLLama-v0", device = "auto", dispatch = True) # See supported models: https://docs.vllm.ai/en/v0.6.4.post1/models/supported_models.html


INFO 06-25 12:37:28 [__init__.py:244] Automatically detected platform cuda.


NameError: name 'InterventionGraph' is not defined

In [50]:
print(llama_model.layers[0].source)

                                     * def forward(
                                     0     self,
                                     1     hidden_states: torch.Tensor,
                                     2     attention_mask: Optional[torch.Tensor] = None,
                                     3     position_ids: Optional[torch.LongTensor] = None,
                                     4     past_key_value: Optional[Cache] = None,
                                     5     output_attentions: Optional[bool] = False,
                                     6     use_cache: Optional[bool] = False,
                                     7     cache_position: Optional[torch.LongTensor] = None,
                                     8     position_embeddings: Optional[tuple[torch.Tensor, torch.Tensor]] = None,  # necessary, but kept here for BC
                                     9     **kwargs: Unpack[FlashAttentionKwargs],
                                    10 ) -> tuple[torch.FloatTensor, O

In [51]:
from nnterp import StandardizedTransformer
from nnterp.nnsight_utils import get_attention
import torch as th
model = StandardizedTransformer("gpt2", device_map="auto")
model.tokenizer.chat_template
with model.trace("a"):
    print(
    get_attention(model, 0).inputs[1].keys())
    print(get_attention(model, 0).input.shape)
    print(th.allclose(model.layers_output[0], get_attention(model, 1).input))

llama_model = StandardizedTransformer("Maykeye/TinyLLama-v0", device_map="auto")
with llama_model.trace("a"):
    print(
    get_attention(llama_model, 0).inputs[1].keys())
    print(get_attention(llama_model, 0).input.shape)
    layer_output = llama_model.layers_output[0]
    attention_input = get_attention(llama_model, 1).input
    attn_hidden_states = get_attention(llama_model, 1).inputs[1]["hidden_states"]
    print(f"hs vs input: {th.allclose(attn_hidden_states, attention_input)}")
    print(th.allclose(layer_output, attention_input))
    print(layer_output)
    print(attention_input)

dict_keys(['past_key_value', 'cache_position', 'attention_mask', 'head_mask', 'use_cache', 'output_attentions'])
torch.Size([1, 1, 768])
False
dict_keys(['hidden_states', 'attention_mask', 'position_ids', 'past_key_value', 'output_attentions', 'use_cache', 'cache_position', 'position_embeddings'])
torch.Size([1, 2, 64])
hs vs input: True
False
tensor([[[ 3.4593e-01, -5.6658e-01,  4.0431e-01,  4.2073e-01, -1.2793e+00,
          -1.6680e-01,  1.5621e-02, -4.5038e-01,  1.4098e-01,  6.4284e-01,
          -6.5602e-01, -2.1935e-01,  1.2947e+00,  2.8273e-01, -2.5297e-01,
          -1.9284e-02, -1.5790e-01,  1.3371e-01, -6.2846e-01, -2.4550e-01,
           7.0501e-01, -2.3435e-01,  1.3520e-01,  8.9260e-01,  1.8702e-01,
          -4.8103e-01, -2.1904e-01,  1.0528e+00,  9.3141e-01, -4.1581e-01,
           3.0944e-01,  3.2379e-01,  5.5943e-01,  1.5155e-01, -1.0615e+00,
          -3.4208e-01, -4.4287e-01, -5.4429e-01,  3.1906e-01, -4.4383e-01,
           8.3231e-01,  1.6720e-01,  1.7813e-01,  7.29

In [30]:
from nnsight import LanguageModel

model = LanguageModel("Maykeye/TinyLLama-v0")
with model.trace(["hello", "hello the fox is jumping"]):
    print(model.model.layers[0].self_attn.source.attention_interface_0.source)
    print(model.model.layers[0].self_attn.source.attention_interface_0.source.torch_nn_functional_scaled_dot_product_attention_0.source)

                                                        * def sdpa_attention_forward(
                                                        0     module: torch.nn.Module,
                                                        1     query: torch.Tensor,
                                                        2     key: torch.Tensor,
                                                        3     value: torch.Tensor,
                                                        4     attention_mask: Optional[torch.Tensor],
                                                        5     dropout: float = 0.0,
                                                        6     scaling: Optional[float] = None,
                                                        7     is_causal: Optional[bool] = None,
                                                        8     **kwargs,
                                                        9 ) -> tuple[torch.Tensor, None]:
 kwargs_get_0                            

NNsightException: 

Traceback (most recent call last):
  File "/tmp/ipykernel_12034/2327001449.py", line 6, in <module>
    print(model.model.layers[0].self_attn.source.attention_interface_0.source.torch_nn_functional_scaled_dot_product_attention_0.source)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/envoy.py", line 1183, in source
    source, line_numbers, fn = inject(fn, wrap, self.name)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/inject.py", line 60, in convert
    source = textwrap.dedent(inspect.getsource(fn))
  File "/usr/lib/python3.10/inspect.py", line 1139, in getsource
    lines, lnum = getsourcelines(object)
  File "/usr/lib/python3.10/inspect.py", line 1121, in getsourcelines
    lines, lnum = findsource(object)
  File "/usr/lib/python3.10/inspect.py", line 940, in findsource
    file = getsourcefile(object)
  File "/usr/lib/python3.10/inspect.py", line 817, in getsourcefile
    filename = getfile(object)
  File "/root/.ven

In [ ]:
from nnterp import StandardizedTransformer

model = StandardizedTransformer("Maykeye/TinyLLama-v0")#, attn_implementation="eager")
# print(model.attentions[0].source)
# print("================"*3)
with model.trace(["hello", "hello the fox is jumping"]):
    print(model.attentions[0].source.attention_interface_0.source.torch_nn_functional_scaled_dot_product_attention_0.source)
    print(model.attentions[0].source.attention_interface_0.output[1].shape)
    print(model.layers_output[0].shape)

NNsightException: 

Traceback (most recent call last):
  File "/tmp/ipykernel_12034/982287227.py", line 7, in <module>
    print(model.attentions[0].source.attention_interface_0.source.torch_nn_functional_scaled_dot_product_attention_0.source)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/envoy.py", line 1183, in source
    source, line_numbers, fn = inject(fn, wrap, self.name)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/inject.py", line 60, in convert
    source = textwrap.dedent(inspect.getsource(fn))
  File "/usr/lib/python3.10/inspect.py", line 1139, in getsource
    lines, lnum = getsourcelines(object)
  File "/usr/lib/python3.10/inspect.py", line 1121, in getsourcelines
    lines, lnum = findsource(object)
  File "/usr/lib/python3.10/inspect.py", line 940, in findsource
    file = getsourcefile(object)
  File "/usr/lib/python3.10/inspect.py", line 817, in getsourcefile
    filename = getfile(object)
  File "/root/.venv/lib/python3

In [25]:
model.config

LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 4,
  "hidden_act": "silu",
  "hidden_size": 64,
  "initializer_range": 0.02,
  "intermediate_size": 256,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 16,
  "num_hidden_layers": 8,
  "num_key_value_heads": 16,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.53.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [5]:
from nnsight import LanguageModel
import torch as th
model = LanguageModel("Maykeye/TinyLLama-v0", device_map="cuda")





You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [3]:
from nnterp.nnsight_utils import skip_layers

def start_from_2(model: LanguageModel, prompt: str, tensor):
    with model.trace(prompt) as trace:
        skip_layers(model, 0, 2, tensor)
        return trace
with model.trace("Hello, world!"):
    layer_0_out = model.model.layers[0].output[0].save()
with start_from_2(model, "Hello, world!", layer_0_out) as trace:
    print(model.output)

AttributeError: __enter__

In [11]:
from nnterp.nnsight_utils import skip_layers, set_layer_output, get_layer_output
import torch as th
with model.trace("Hello, world!"):
    out = model.lm_head.output.save()
with model.trace("Hello, world!"):
    skip_layers(model, 1, 5)
    skip_out = model.lm_head.output.save()
with model.trace("Hello, world!"):
    set_layer_output(model, 5, get_layer_output(model, 0))
    old_skip_out = model.lm_head.output.save()

assert not th.allclose(out, skip_out)
assert th.allclose(old_skip_out, skip_out)

In [13]:
from nnsight import LanguageModel
model = LanguageModel("Maykeye/TinyLLama-v0")
with model.trace("hello"):
    input = model.model.layers[0].input
    model.model.layers[0].skip((input,))
print("First method succeeded")
with model.trace("hello"):
    input = model.model.layers[0].input
    model.model.layers[0].skip(input)
print("Second method succeeded")

First method succeeded


NNsightException: 

Traceback (most recent call last):
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/backends/execution.py", line 21, in __call__
    tracer.execute(fn)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/tracing/tracer.py", line 331, in execute
    self.model.interleave(interleaver, self.fn, *args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/modeling/mixins/meta.py", line 76, in interleave
    return super().interleave(interleaver, fn, *args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/envoy.py", line 705, in interleave
    interleaver(fn, *args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/interleaver.py", line 310, in __call__
    fn(*args, **kwargs)
  File "/root/.venv/lib/python3.10/site-packages/nnsight/intervention/envoy.py", line 372, in __call__
    else self._module(*args, **kwargs)
  File "/root/.venv/lib/python3.10/site-pack

In [23]:
rename = {".*attn": "self_attn"}
# broken_model = LanguageModel("gpt2", rename=rename)
print(broken_model)
with broken_model.trace("hello"):
    out = broken_model.lm_head.output.save()
    out_broken = broken_model.transformer.h[9].self_attn.c_attn.output.save()
print(out.shape)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (self_attn): GPT2SdpaAttention(
          (self_attn): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (generator): Generator(
    (streamer): Streamer()
  )
)


AttributeError: 'Conv1D' object has no attribute 'output'

In [18]:
model = LanguageModel("gpt2")
with model.trace("hello"):
    out_normal = model.lm_head.output.save()
from torch.testing import assert_close
assert_close(out_normal, out)

In [12]:
attention_names = [r".*\.attn$", r".*\.self_attention$", r".*\.attention$"]
model_names = [r".*\.transformer$", r".*\.gpt_neox$"]
layer_names = [r".*\.h$"]
ln_names = [r".*\.final_layer_norm$", r".*\.ln_f$"]
lm_head_names = [r".*\.embed_out$"]


llm_rename_dict = {}
for name in attention_names:
    llm_rename_dict[name] = "self_attn"
for name in model_names:
    llm_rename_dict[name] = "model"
for name in layer_names:
    llm_rename_dict[name] = "layers"
for name in ln_names:
    llm_rename_dict[name] = "ln_final"
for name in lm_head_names:
    llm_rename_dict[name] = "lm_head"

print(llm_rename_dict)
patched_model = LanguageModel("gpt2", rename=llm_rename_dict)
print(patched_model)
# print(LanguageModel("gpt2"))

{'.*\\.attn$': 'self_attn', '.*\\.self_attention$': 'self_attn', '.*\\.attention$': 'self_attn', '.*\\.transformer$': 'model', '.*\\.gpt_neox$': 'model', '.*\\.h$': 'layers', '.*\\.final_layer_norm$': 'ln_final', '.*\\.ln_f$': 'ln_final', '.*\\.embed_out$': 'lm_head'}
GPT2LMHeadModel(
  (model): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (self_attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)


In [11]:
LanguageModel("gpt2")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (generator): Generator(
    (streamer): Streamer()
  )
)

In [7]:
LanguageModel("meta-llama/Meta-Llama-3-8B-Instruct")

c:\Users\Travail\Documents\repos\nnterp\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Travail\.cache\huggingface\hub\models--meta-llama--Meta-Llama-3-8B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

## My branch

In [3]:
from nnterp import load_model
patched_model = load_model('gpt2', use_model_renaming=True)
patched_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (self_attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,)

In [4]:
print(patched_model.model)
print()
print(patched_model.model.layers[0])
print()
print(patched_model.model.layers[0].self_attn)

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (self_attn): GPT2SdpaAttention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (layers): ModuleList(
    (0-11): 12 x GPT2Block(


In [6]:
import torch
with patched_model.trace("hello"):
    out = patched_model.model.layers[0].output[0].save()
with patched_model.trace("hello"):
    out_old = patched_model.transformer.h[0].output[0].save()
torch.allclose(out, out_old)

True

In [7]:
del patched_model._envoy.transformer
with patched_model.trace("hello"):
    out = patched_model.model.layers[0].output[0].save()
try:
    with patched_model.trace("hello"):
        out_old = patched_model.transformer.h[0].output[0].save()
    torch.allclose(out, out_old)
except Exception as e:
    print(e)

'GPT2Block' object has no attribute 'output'


In [9]:
del patched_model._model.transformer
try:
    with patched_model.trace("hello"):
        out = patched_model.model.layers[0].output[0].save()
except Exception as e:
    print(e)
print("-"*100)
try:
    with patched_model.trace("hello"):
        out_old = patched_model.transformer.h[0].output[0].save()
    torch.allclose(out, out_old)
except Exception as e:
    print(e)


'GPT2LMHeadModel' object has no attribute 'transformer'
----------------------------------------------------------------------------------------------------
'GPT2LMHeadModel' object has no attribute 'transformer'


In [4]:
model = load_model('gpt2', use_model_renaming=False)
model

/dlabscratch1/cdumas/nnterp/.conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (generator): WrapperModule()
)

In [11]:
type(model._envoy.transformer.h)

nnsight.envoy.Envoy

## others

In [2]:
load_model("bigscience/bloom-7b1")

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 4096)
    (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )


In [4]:
load_model("EleutherAI/pythia-160m")

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
      